# Get weights from Fusion

In [1]:
cd /project2/mstephens/causalTWAS/fusion_weights

In [1]:
wget https://data.broadinstitute.org/alkesgroup/FUSION/WGT/YFS.BLOOD.RNAARR.tar.bz2

--2019-10-29 16:14:53--  https://data.broadinstitute.org/alkesgroup/FUSION/WGT/YFS.BLOOD.RNAARR.tar.bz2
Resolving data.broadinstitute.org (data.broadinstitute.org)... 69.173.92.29
Connecting to data.broadinstitute.org (data.broadinstitute.org)|69.173.92.29|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 69800449 (67M) [application/x-bzip2]
Saving to: ‘YFS.BLOOD.RNAARR.tar.bz2’

100%[======================================>] 69,800,449  3.46MB/s   in 14s    

2019-10-29 16:15:08 (4.75 MB/s) - ‘YFS.BLOOD.RNAARR.tar.bz2’ saved [69800449/69800449]



In [3]:
tar xjf YFS.BLOOD.RNAARR.tar.bz2

In [2]:
ls

YFS.BLOOD.RNAARR       YFS.BLOOD.RNAARR.pos	 YFS.BLOOD.RNAARR.profile.err
YFS.BLOOD.RNAARR.list  YFS.BLOOD.RNAARR.profile  YFS.BLOOD.RNAARR.tar.bz2


# Process WTCCC data

WTCCC data obtained from Peter (`bd.RData`). We will prepare genotype file in BIMBAM format and relatedness matrix file.

In [3]:
# sinteractive --time=1-12:00:00 --partition=mstephens --mem=20G
module load R/3.5.1

In [15]:
cd /project2/mstephens/causalTWAS/causal-TWAS/code

In [33]:
Rscript run_WTCCC_data_process.R

In [17]:
ls /project2/mstephens/causalTWAS/WTCCC

bd.geno.txt.gz	bd.pheno.txt  bd.QC.RData  bd.RData


# Gene annotation data

In [11]:
mkdir /project2/mstephens/causalTWAS/annotation

In [21]:
cd /project2/mstephens/causalTWAS/annotation; wget http://hgdownload.soe.ucsc.edu/goldenPath/hg17/database/refGene.txt.gz

--2019-11-08 09:56:18--  http://hgdownload.soe.ucsc.edu/goldenPath/hg17/database/refGene.txt.gz
Resolving hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)|128.114.119.163|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7550681 (7.2M) [application/x-gzip]
Saving to: ‘refGene.txt.gz’

100%[======================================>] 7,550,681   6.66MB/s   in 1.1s   

2019-11-08 09:56:19 (6.66 MB/s) - ‘refGene.txt.gz’ saved [7550681/7550681]



In [22]:
gunzip refGene.txt.gz

In [ ]:
Rscript /project2/mstephens/causalTWAS/causal-TWAS/code/gene_annotation.R refGene.txt 500000

In [24]:
ls

refGene_processed_5e+05.txt  refGene.txt
